In [1]:
# This let's me iterate faster by deleting it at the beginning
import os
os.remove('../firstpage.json')

# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess

class EssayItem(scrapy.Item):
    subject = scrapy.Field()
    title = scrapy.Field()
    text = scrapy.Field()

class SESpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "SES" # for student essay spider
    
    # URL(s) to start with.
    start_urls = [
        'https://www.ukessays.com/essays/',
    ]

    # Use XPath to parse the response from the start_urls we declared.
    def parse(self, response):

        # Iterate over every column element on the page.
        for subjects in response.xpath('//div[@class="col-lg-3 col-sm-6"]/div/ul/li'):
                
            # Create our subject from this top-level page
            subject = subjects.xpath('a/text()').extract_first()
            
            # Create an EssayItem called essay_item, empty for now
            essay_item = EssayItem()
            
            # Create new url for parse_essays to use
            essay_list_url = response.urljoin(subjects.xpath('a/@href').extract_first())
            
            # New request with essay_list_url, pass in our essay_item
            yield scrapy.Request(essay_list_url, callback=self.parse_essays, 
                                 dont_filter=True,
                                 meta={'item':essay_item,
                                       'subject':subject})
            
    def parse_essays(self, response):

        # Retrieve essay items from metadata
        essay_item = response.request.meta['item']
        subject = response.request.meta['subject']
        
        for i, essay in enumerate(response.xpath('//div/article/div/h4')):
            # Don't break the nice people's website
            if i > 5:
                break
                
            # Xpath to the essay's title
            title = essay.xpath('a/text()').extract_first()
            
            # Xpath into the actual essay's link... finally!
            essay_url = response.urljoin(essay.xpath('a/@href').extract_first())
            
            # This one should return the actual text, along with the rest of the item fields
            yield scrapy.Request(essay_url, callback=self.collect_essay,
                                 dont_filter=False,
                                 meta={'item':essay_item,
                                       'subject':subject,
                                       'title':title})
        
    def collect_essay(self, response):
        # Retrieve our essay_item, once again, from Response's metadata
        essay_item = response.request.meta['item']
                
        # Add the text, subject, and title into our essay_item
        # Something weird was happening, probably due to the aysync nature of this stuff
        essay_item['text'] = response.xpath('//body/main/div/div/article/p/text()').extract()
        essay_item['subject'] = response.request.meta['subject']
        essay_item['title'] = response.request.meta['title']        
        
        yield essay_item
        
        
        
# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': '../firstpage.json',  # Name our storage file.
    'LOG_ENABLED': False,           # Turn off logging for now.
    'AUTOTHROTTLE_ENABLED' : False,
    'HTTPCACHE_ENABLED' : False,
    'ROBOTSTXT_ENABLED' : False,
    'DOWNLOAD_DELAY' : 1.5
})

# Start the crawler with our spider.
process.crawl(SESpider)
process.start()
print('Success!')

import pandas as pd

uk_students = pd.read_json('../firstpage.json', orient='records', encoding='latin')
uk_students['student'] = 1
print(uk_students.shape)
uk_students.head()

Success!
(675, 4)


,subject,text,title,student
0,Ecommerce,"[\n, \n, \n, \n, \n, Face of consumerism throu...",Impact of E-commerce on the Hospitality Industry,1
1,Drama,"[\n, \n, \n, \n, \n, ped importance, price el...",Importance of Elasticity Demand,1
2,Design,"[\n, \n, \n, \n, \n, To export a reference to ...",Posters of the Second World War - An Analysis,1
3,Data Analysis,"[\n, \n, \n, \n, \n, When we talk about the Ma...",Mathematical description of OFDM,1
4,Cultural Studies,"[\n, \n, \n, \n, \n, The culture of the educat...",Cultural Analysis of Pakistan,1


In [3]:
import pickle

pickle.dump(uk_students, open('../uk_students.pkl', 'wb'))

Woo!! All done with that one. On the the harder part: finding proffessional samples. The websites I've found generally do not post very many samples, unlike this UK Student Sample Bank. There were tens of thousands of essays to choose from, but the same website only posted 40 or so samples from their professionals. I will have to manually find a bunch of websites and make scrapers for all of them :(